<a href="https://colab.research.google.com/github/Jagadish19k/retail-sales-forecasting/blob/main/Capstone_Jagadish_kumar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Capstone Project — Retail Sales Insights & Demand Forecasting**

**Author:** Jagadish Kumar  
**Date:** 2025-11-30

This notebook implements the full workflow: data loading, cleaning, EDA,
statistical tests, regression modeling, time-series forecasting, and
PowerPoint generation. Visuals and outputs are saved to `/content`.


# 🟦 Day 1 — Data Loading, Merging & Cleaning

### 🔹 1.1 Import Libraries  
### 🔹 1.2 Load CSV Files  
### 🔹 1.3 Merge train + stores + features  
### 🔹 1.4 Handle Missing Values  
### 🔹 1.5 Feature Engineering (Year, Month, Week, Holiday Flag)  
### 🔹 1.6 Save Cleaned Dataset  


In [1]:
# Setup & imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

%matplotlib inline
plt.rcParams['figure.figsize'] = (10,5)

DATA_DIR = Path('/content')  # Colab working directory
TRAIN_CSV = DATA_DIR / 'train.csv'
STORES_CSV = DATA_DIR / 'stores.csv'
FEATURES_CSV = DATA_DIR / 'features.csv'

print("Data path checks:", TRAIN_CSV.exists(), STORES_CSV.exists(), FEATURES_CSV.exists())


Data path checks: False False False


In [2]:
# Load datasets
train = pd.read_csv(TRAIN_CSV, parse_dates=['Date'])
stores = pd.read_csv(STORES_CSV)
features = pd.read_csv(FEATURES_CSV, parse_dates=['Date'])

print("Shapes ->", train.shape, stores.shape, features.shape)
display(train.head())


FileNotFoundError: [Errno 2] No such file or directory: '/content/train.csv'

In [ ]:
# Merge and basic cleaning
df = train.merge(stores, on='Store', how='left').merge(features, on=['Store','Date'], how='left')
print("Merged shape:", df.shape)

# Ensure Date is datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Fill numeric missing with median
num_cols = df.select_dtypes(include=['number']).columns.tolist()
for c in num_cols:
    if df[c].isnull().any():
        df[c] = df[c].fillna(df[c].median())

# Fill object missing with 'Unknown'
for c in df.select_dtypes(include=['object']).columns.tolist():
    df[c] = df[c].fillna('Unknown')

# Derived features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.isocalendar().week

# Ensure holiday flag exists
if 'IsHoliday' not in df.columns:
    if 'Holiday_Flag' in df.columns:
        df['IsHoliday'] = df['Holiday_Flag'].astype(int)
    else:
        df['IsHoliday'] = 0
else:
    df['IsHoliday'] = df['IsHoliday'].astype(int)

# Save cleaned dataset
out_clean = DATA_DIR / 'Cleaned_Merged.csv'
df.to_csv(out_clean, index=False)
print("Saved cleaned data to", out_clean)
display(df.head())


# 🟩 Day 2 — Exploratory Data Analysis (EDA)

### 🔹 2.1 Top 10 Stores by Sales  
### 🔹 2.2 Monthly Sales Trend  
### 🔹 2.3 Correlation Analysis  
### 🔹 2.4 Holiday vs Non-Holiday Boxplot  
### 🔹 2.5 Add Insights Below Each Chart  


In [ ]:
# Top 10 stores by total Weekly_Sales
if 'Weekly_Sales' in df.columns:
    top_stores = df.groupby('Store')['Weekly_Sales'].sum().sort_values(ascending=False).head(10)
    display(top_stores)
    fig, ax = plt.subplots()
    top_stores.plot(kind='bar', ax=ax)
    ax.set_title('Top 10 Stores by Total Weekly Sales')
    ax.set_ylabel('Total Weekly Sales')
    fig.savefig(DATA_DIR/'top_10_stores.png', bbox_inches='tight')
    plt.show()

# Monthly chain-level trend
if 'Weekly_Sales' in df.columns:
    monthly = df.set_index('Date').resample('M')['Weekly_Sales'].sum()
    fig, ax = plt.subplots()
    monthly.plot(ax=ax)
    ax.set_title('Monthly Sales Trend (Chain-level)')
    ax.set_ylabel('Total Weekly Sales')
    fig.savefig(DATA_DIR/'monthly_sales_trend.png', bbox_inches='tight')
    plt.show()


In [ ]:
# Correlation matrix (numeric)
numeric = df.select_dtypes(include=['number'])
if numeric.shape[1] > 1:
    corr = numeric.corr()
    fig, ax = plt.subplots(figsize=(10,8))
    cax = ax.matshow(corr.fillna(0))
    fig.colorbar(cax)
    ax.set_xticks(range(len(corr.columns)))
    ax.set_yticks(range(len(corr.columns)))
    ax.set_xticklabels(corr.columns, rotation=90)
    ax.set_yticklabels(corr.columns)
    ax.set_title('Correlation matrix (numeric)')
    fig.savefig(DATA_DIR/'correlation_matrix.png', bbox_inches='tight')
    plt.show()

# Holiday vs non-holiday boxplot
if 'Weekly_Sales' in df.columns:
    fig, ax = plt.subplots(figsize=(8,4))
    df.boxplot(column='Weekly_Sales', by='IsHoliday', ax=ax)
    ax.set_title('Weekly Sales: Holiday (1) vs Non-Holiday (0)')
    ax.set_xlabel('IsHoliday')
    ax.set_ylabel('Weekly_Sales')
    plt.suptitle('')
    fig.savefig(DATA_DIR/'holiday_boxplot.png', bbox_inches='tight')
    plt.show()


# 🟨 Day 3 — Statistical Analysis

### 🔹 3.1 Holiday vs Non-Holiday (t-test & Mann–Whitney)  
### 🔹 3.2 Store-Level Aggregated t-test  
### 🔹 3.3 Interpretation of Results  


In [ ]:
from scipy import stats

stat_results = {}

# Count rows
s_h = df[df['IsHoliday']==1]['Weekly_Sales']
s_nh = df[df['IsHoliday']==0]['Weekly_Sales']

# t-test (safe)
if len(s_h)>1 and len(s_nh)>1:
    t_stat, p_val = stats.ttest_ind(s_h, s_nh, equal_var=False)
    stat_results['t_test'] = {'t':float(t_stat),'p':float(p_val)}
else:
    stat_results['t_test'] = "Not enough data"

# Aggregated store-level t-test
agg = df.groupby(['Store','IsHoliday'])['Weekly_Sales'].mean().reset_index()
agg_h = agg[agg['IsHoliday']==1]['Weekly_Sales']
agg_nh = agg[agg['IsHoliday']==0]['Weekly_Sales']

if len(agg_h)>1 and len(agg_nh)>1:
    t2, p2 = stats.ttest_ind(agg_h, agg_nh, equal_var=False)
    stat_results['aggregated_ttest'] = {'t':float(t2),'p':float(p2)}
else:
    stat_results['aggregated_ttest'] = "Not enough aggregated data"

# Mann–Whitney U test
if len(s_h)>0 and len(s_nh)>0:
    u, p = stats.mannwhitneyu(s_h, s_nh)
    stat_results['mann_whitney'] = {'u':float(u),'p':float(p)}
else:
    stat_results['mann_whitney'] = "Not enough samples"

stat_results


# 🟧 Day 4 — Predictive Modeling & Forecasting

### 🔹 4.1 Regression Model  
### 🔹 4.2 R², MAE, RMSE  
### 🔹 4.3 Top Feature Coefficients  
### 🔹 4.4 Holt–Winters 12-Week Forecast  
### 🔹 4.5 Save Forecast Plot  


In [ ]:
# Regression model to predict Weekly_Sales
model_metrics = {}

if 'Weekly_Sales' in df.columns:
    X = df.select_dtypes(include=['number']).drop(columns=['Weekly_Sales'], errors='ignore')
    y = df['Weekly_Sales'].fillna(0)

    if X.shape[1] > 0:
        X_train, X_test, y_train, y_test = train_test_split(
            X.fillna(0), y, test_size=0.2, random_state=42
        )

        lr = LinearRegression().fit(X_train, y_train)
        y_pred = lr.predict(X_test)

        # Compute RMSE manually (for older sklearn)
        mse = mean_squared_error(y_test, y_pred)
        rmse = mse ** 0.5

        model_metrics = {
            'R2': float(r2_score(y_test, y_pred)),
            'MAE': float(mean_absolute_error(y_test, y_pred)),
            'RMSE': float(rmse)
        }

        # Save top 10 coefficients
        coef_series = pd.Series(lr.coef_, index=X.columns).abs().sort_values(ascending=False)
        coef_series.head(10).to_csv('/content/model_top_coefs.csv')

        print("Model Metrics:", model_metrics)
    else:
        print("No numeric features available for regression")


In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

ts = df.set_index('Date').resample('W')['Weekly_Sales'].sum().dropna().sort_index()

if len(ts) >= 52:
    model = ExponentialSmoothing(ts, seasonal='add', seasonal_periods=52).fit()
    fc = model.forecast(12)
else:
    # fallback if data insufficient
    last_val = ts.iloc[-1]
    fc = pd.Series([last_val]*12, index=pd.date_range(ts.index[-1], periods=12, freq='W'))

forecast_df = fc.reset_index()
forecast_df.columns = ['Date','Forecast_Weekly_Sales']
forecast_df.to_csv('/content/ts_forecast.csv', index=False)

# Plot forecast
fig, ax = plt.subplots(figsize=(10,4))
ts.plot(ax=ax, label='History')
fc.plot(ax=ax, label='Forecast')
ax.set_title('Weekly Sales Forecast (12 Weeks)')
ax.legend()

plt.show()
print("Forecast saved to ts_forecast.csv")
display(forecast_df.head())


In [ ]:
import json

final_stats = {
    'model_metrics': model_metrics,
    'stat_tests': stat_results,
    'generated_on': datetime.utcnow().isoformat()
}

with open('/content/model_metrics.json','w') as f:
    json.dump(final_stats, f, indent=2)

print("Saved model metrics to model_metrics.json")


# 🟫 Day 5 — Insights, Recommendations & PPT Generation

### 🔹 5.1 Key Insights  
### 🔹 5.2 Executive Summary  
### 🔹 5.3 Final Recommendations  
### 🔹 5.4 Generate Enhanced PPT with Images  
### 🔹 5.5 Submission Checklist  


In [ ]:
# install python-pptx (needed to generate PPTX)
!pip install python-pptx


In [ ]:
# Make sure to run: !pip install python-pptx before this cell

try:
    from pptx import Presentation
    from pptx.util import Inches

    prs = Presentation()

    # Title slide
    slide = prs.slides.add_slide(prs.slide_layouts[0])
    slide.shapes.title.text = "Capstone Project: Sales Insights & Forecasting"
    slide.placeholders[1].text = f"Author: Jagadish Kumar\nGenerated: {datetime.utcnow().strftime('%Y-%m-%d %H:%M UTC')}"

    # Key Metrics slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    slide.shapes.title.text = "Key Model Metrics"
    tf = slide.shapes.placeholders[1].text_frame
    tf.text = str(model_metrics)

    # Add EDA images
    images = [
        'top_10_stores.png',
        'monthly_sales_trend.png',
        'correlation_matrix.png',
        'holiday_boxplot.png',
        'ts_forecast_plot.png'
    ]

    for img in images:
        path = Path('/content')/img
        if path.exists():
            slide = prs.slides.add_slide(prs.slide_layouts[5])
            slide.shapes.title.text = img.replace('_',' ').title()
            slide.shapes.add_picture(str(path), Inches(1), Inches(1.5), height=Inches(4.5))

    # Summary slide
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    slide.shapes.title.text = "Executive Summary"
    tf = slide.shapes.placeholders[1].text_frame
    tf.text = (
        "• Identified store performance patterns\n"
        "• Evaluated holiday vs non-holiday sales\n"
        "• Built regression model for weekly sales\n"
        "• Produced 12-week demand forecast\n"
        "• Recommendations: inventory planning, staffing, promotions"
    )

    ppt_path = "/content/Capstone_Report.pptx"
    prs.save(ppt_path)
    print("PowerPoint saved to:", ppt_path)

except Exception as e:
    print("PPTX creation failed:", e)
